# Notebook for generate Quantitative Graphics

### Libraries import

In [1]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy import stats
import matplotlib.pyplot as plt

sys.path.append("../../")

2022-08-08 22:25:18.145914: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
from utils.common import format_index
from utils.metrics import brownForsythe_test, levene_test, bartlett_test
from utils.metrics import accuracy, precision, recall, specificity, f1_score
from utils.metrics import mannWhitney_test, kruskalWallis_test, kolmogorovSmirnov_test
from utils.metrics import dagostinoPearson_test, andersonDarling_test, shapiroWilks_test, chiSquare_test, fOneWay_test

In [3]:
def precision_recall_curve(y_true, y_pred, num_thresholds=200):
    precisions = []
    recalls = []
    thresholds = np.linspace(np.min(y_pred), np.max(y_pred), num_thresholds)
    for t in thresholds:
        tp = np.count_nonzero(np.logical_and(y_true, (y_pred > t)))
        fp = np.count_nonzero(np.logical_and(np.logical_not(y_true), (y_pred > t)))
        fn = np.count_nonzero(np.logical_and(y_true, (y_pred <= t)))
        if tp+fp == 0:
            precisions.append(0)
        else:
            precisions.append(precision(tp, fp))
        if tp + fn == 0:
            recalls.append(0)
        else:
            recalls.append(recall(tp, fn))
    return np.r_[precisions], np.r_[recalls], thresholds

### Experiment selection

In [4]:
experiment_id = "0006"
root_path = "../../results/Ganomaly_3D/"
for i in sorted(os.listdir(root_path)):
    if experiment_id in i:
        experiment_folder = os.path.join(root_path, i)
experiment_folder

'../../results/Ganomaly_3D/0006_Ganomaly3D-64x64x64x1'

### Errors loading

In [5]:
base_path = os.path.join(experiment_folder, "outputs/errors/")
for t in ["encoder", "contextual", "adversarial"]:
        for c in ["normal", "abnormal"]:
            globals()["all_{}_{}".format(t, c)] = np.r_[[]]

for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "val", "test"]:
        if m == "train":
            if os.path.isfile(os.path.join(base_path, t, m, "normal.npy")):
                classes = ["normal"] 
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]

        for c in classes:
            all_data = "all_{}_{}".format(t, c)
            errors = np.load(os.path.join(base_path, t, m, c + ".npy"))
            globals()["{}_{}_{}".format(m, t, c)] = errors
            globals()[all_data] = np.concatenate([globals()[all_data], errors])

### Errors by patients loading

In [6]:
base_path = os.path.join(experiment_folder, "outputs/latent_vectors/input_generator")
for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "val", "test"]:
        if m == "train":
            if "train_encoder_normal" in globals().keys():
                classes = ["normal"] 
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]
        for c in classes:
            patients_ids_positions = [
                int(i.split("_")[1].split("-")[1].split(".")[0]) for i in sorted(
                    os.listdir(os.path.join(base_path, m, c))
                )
            ]
            data = "{}_{}_{}".format(m, t, c)
            key = "{}_{}".format(data, "patients")
            globals()[key] = {}

            for p_id in np.unique(patients_ids_positions):
                globals()[key][p_id] = []

            for i, p_id in enumerate(patients_ids_positions):
                globals()[key][p_id].append(globals()[data][i])

### Precision vs Recall

In [8]:
errors = ["normal", "abnormal"]
for group in ["encoder", "contextual", "adversarial"]:
    data = "val_{}_".format(group)
    y_true = np.concatenate([[i]*globals()[data + j].shape[0] for i,j in enumerate(errors)]) 
    y_pred = np.concatenate([globals()[data+i] for i in errors])

    precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred)
    deltas_pre_4_rec = np.abs(precisions - recalls)
    threshold = thresholds[np.argmin(deltas_pre_4_rec[deltas_pre_4_rec != 0])]

    path = os.path.join(experiment_folder, "outputs/graphics/quantitative/")
    plt.plot(thresholds, precisions, label="Precision")
    plt.plot(thresholds, recalls, label="Recall")
    plt.axvline(threshold, color="black", alpha=0.5)
    plt.title('Precision and Recall for different thresholds in Validation of {} error'.format(group))
    plt.xlabel('Threshold')
    plt.ylabel('Precision/Recall')
    plt.legend()
    plt.savefig(os.path.join(path, "Precision_vs_Recall_{}.png".format(group)),dpi=1200)
    plt.close()